In [ ]:
import requests
import tiktoken
import json
import re

# Replace with your OpenRouter API key
API_KEY = 'urapikey'
API_URL = 'https://openrouter.ai/api/v1/chat/completions'

# Define the headers for the API request
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# Initialize the conversation history
conversation_history = []

# Load the tokenizer (use the appropriate encoding for your model)
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    """Count the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def read_txt_file(file_path):
    """Read the contents of the .txt file and return it as a string."""
    with open(file_path, 'r', encoding="utf-8") as file:
        file_content = file.read()
    return file_content

def extract_json_from_text(text):
    """Extract JSON-formatted data from text using regex."""
    match = re.search(r'```json\n(.*?)\n```', text, re.DOTALL)
    if match:
        json_text = match.group(1)
        try:
            return json.loads(json_text)  # Convert string to JSON object
        except json.JSONDecodeError:
            print("Error: Extracted JSON is invalid.")
            return None
    else:
        print("No JSON found in the response.")
        return None

def chat_with_deepseek(user_input):
    global conversation_history

    # Read the .txt file if provided
    file_content = read_txt_file("resume_files\\try.txt")

    # Combine the user's input and the file content into a single prompt
    combined_input = "this is an extracted text of a resume " + file_content + "\n" + user_input

    # Append the user's combined message to the conversation history
    conversation_history.append({"role": "user", "content": combined_input})

    # Define the request payload (data) with the conversation history
    data = {
        "model": "deepseek/deepseek-chat:free",
        "messages": conversation_history
    }

    # Send the POST request to the DeepSeek API
    response = requests.post(API_URL, json=data, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the response JSON
        response_data = response.json()
        
        # Extract the assistant's response
        assistant_response = response_data['choices'][0]['message']['content']

        # Extract JSON data from the assistant's response
        extracted_json = extract_json_from_text(assistant_response)

        if extracted_json:
            print("Extracted JSON Data:")
            print(json.dumps(extracted_json, indent=4, ensure_ascii=False))  # Pretty print JSON
            
            # Save to a JSON file
            with open("extracted_resume.json", "w", encoding="utf-8") as json_file:
                json.dump(extracted_json, json_file, indent=4, ensure_ascii=False)

            print("JSON data saved successfully as 'extracted_resume2.json'.")
        else:
            print("No valid JSON found in the response.")

    else:
        print("Failed to fetch data from API. Status Code:", response.status_code)


In [ ]:
chat_with_deepseek('''{
  "full_name": "string",
  "phone_number": "string",
  "email": "string",
  "residence": "string",
  "linkedin": "string",
  "github": "string",
  "about_me": "string",
  "personal_projects": [
    {"name": "string", "description": "string", "date": "string"}
  ],
  "academic_projects": [
    {"name": "string", "description": "string", "date": "string"}
  ],
  "professional_projects": [
    {"name": "string", "description": "string", "date": "string"}
  ],
  "education": {
    "licence": [
      {"name": "string", "description": "string", "date": "string"}
    ],
    "master": [
      {"name": "string", "description": "string", "date": "string"}
    ],
    "phd": [
      {"name": "string", "description": "string", "date": "string"}
    ]
  },
  "engineering_entreprise_work": [
    {"job_title": "string","entreprise":"string", "description": "string", "start_date": "string", "end_date": "string"}
  ],
  "non_engineering_work": [
    {"name": "string", "description": "string", "start_date": "string", "end_date": "string"}
  ],
  "certifications": [
    {"name": "string", "description": "string", "date": "string"}
  ],
  "extra_curricular_activities": [
    {"name": "string", "description": "string", "date": "string"}
  ],
  "skills": [
    "string"
  ],
  "books": [
    "string"
  ],
  "languages": [
    "string"
  ]
}



  
''')

Failed to fetch data from API. Status Code: 401
